In [24]:

import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import os
from sklearn.model_selection import train_test_split
import json
from math import sin, cos
import imutils
from skimage.transform import rotate

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms.autoaugment import AutoAugmentPolicy
from skimage.transform import rotate
from skimage.io import imread, imsave, imshow
from skimage.transform import rescale, resize, downscale_local_mean


import cv2
from PIL import Image

import matplotlib.pyplot as plt
from IPython.display import clear_output

In [4]:
def rot_mat(alpha):
    res = np.zeros((2, 2))
    res[0,0] = np.cos(alpha/180*np.pi)
    res[0,1] = -np.sin(alpha/180*np.pi)
    res[1,0] = np.sin(alpha/180*np.pi)
    res[1,1] = np.cos(alpha/180*np.pi)
    return res

In [31]:
orig = cv2.imread('data/new_orig_03.png')

In [42]:
N=10496

In [53]:
angles = [a for a in range(0, 90, 2)]+[a for a in range(270, 360, 2)]+[a for a in range(90, 270, 4)]
number_each_angle = 4

In [54]:
ind = 1201

In [55]:
np.random.seed(59)

In [56]:


for angle in tqdm(angles):
    for j in range(number_each_angle):
        c_y_c_x = np.random.randint(512*2**0.5+1500, N-512*2**0.5-800, 2)
        c_y, c_x = c_y_c_x[0], c_y_c_x[1]
        patch_big = orig[c_y-725:c_y+725,c_x-725:c_x+725]
#         print(c_y_c_x)
#         print(patch_big.shape)
        new_patch = (rotate(patch_big, angle)[725-512:2*725-(725-512), 725-512:2*725-(725-512), :]*255).astype('int')
        #print(new_patch)
        corners = []
        y1_x1 =  (c_y_c_x+np.array([int(-512), int(-512)])@rot_mat(angle)).astype('int')
        y2_x2 =  (c_y_c_x+np.array([int(-512), int(-512)])@rot_mat(angle+90)).astype('int')
        y3_x3 =  (c_y_c_x+np.array([int(-512), int(-512)])@rot_mat(angle+180)).astype('int')
        y4_x4 =  (c_y_c_x+np.array([int(-512), int(-512)])@rot_mat(angle+270)).astype('int')
        corners.append(y1_x1)
        corners.append(y2_x2)
        corners.append(y3_x3)
        corners.append(y4_x4)
        left_top = sorted(corners, key = lambda x: (x[1], x[0]))[0]
        right_bottom = sorted(corners, key = lambda x: (x[1], x[0]), reverse=True)[0]
        left_bottom = (np.array([c_y, c_x]) + np.array(left_top - c_y_c_x)@rot_mat(90)).astype('int16')
        right_top = (np.array([c_y, c_x]) + np.array(right_bottom - c_y_c_x)@rot_mat(90)).astype('int16')
        left_top = left_top.tolist()
        right_top = right_top.tolist()
        left_bottom = left_bottom.tolist()
        right_bottom = right_bottom.tolist()
        left_top.reverse()
        right_top.reverse()
        left_bottom.reverse()
        right_bottom.reverse()
        res = {
        'left_top': left_top,
        'right_top': right_top,
        'left_bottom': left_bottom,
        'right_bottom': right_bottom,
        'angle': angle
        }
        cv2.imwrite(f'data/train_ext/img/{ind}.png', new_patch)
        with open(f'data/train_ext/json/{ind}.json', 'w') as f:
            json.dump(res, f)
        ind += 1

100%|████████████████████████████████████████████████████████████████████████████████| 135/135 [05:35<00:00,  2.48s/it]
